import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480    #80
cls_num=2
btch=20
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=ProximalPhalanxOutlineCorrect(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg0)
print(0,len(vars()['ecg'+str(0)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(0)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (600, 81)
first_test_shape= (291, 81)
classes_quantity= 2
tr_lbls=	 {0, 1}
Count_labels= [406]
max(train_feature_Altitude)= 1.9029
min(train_feature_Altitude)= -1.4834
first_train_sample=
 [ 1.       -0.70673  -0.65214  -0.45675  -0.21008   0.075267  0.34994
  0.59501   0.77936   1.0623    1.3086    1.4932    1.5211    1.5118
  1.3693    1.169     0.9772    0.71438   0.49046   0.23282  -0.054244
 -0.31903  -0.57195  -0.72026  -0.72978  -0.6763   -0.4762   -0.22279
  0.044653  0.33074   0.62671   0.90485   1.1938    1.4955    1.6582
  1.6582    1.5609    1.5457    1.5377    1.5915    1.6089    1.6576
  1.4566    1.1623    0.86912   0.59661   0.29749   0.034627 -0.21209
 -0.47783  -0.65511  -0.57193  -0.65446  -0.75273  -0.92547  -1.0641
 -1.218    -1.33     -1.3396   -1.2731   -1.1312   -0.98085  -0.84127
 -0.74757  -0.64479  -0.62498  -0.6119   -0.70658  -0.7906   -0.9123
 -1.0589   -1.1936   -1.3203   -1.391    -1.3003   -1.1547   -1.0317
 -0.8765   -0.75764  -0.607

In [3]:
np.shape(ecg)

(600, 481)

In [4]:
ecg[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
0 194	1 406	
 max = 406


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (0,cls_num):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  0 train shape =  (194, 481)
Up to class  1 train shape =  (600, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[194 406]


In [8]:
xtrain[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (0,cls_num):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 0  >> 
max magnitude class 0  =  1.7743120350766923
min magnitude class 0  =  -1.299820826698177
after normalizing >>
max magnitude class 0  =  0.9194507121403934
min magnitude class 0  =  -0.8879297726404382

 cls 1  >> 
max magnitude class 1  =  1.8237000000000003
min magnitude class 1  =  -1.4568201800265863
after normalizing >>
max magnitude class 1  =  0.9484874675439612
min magnitude class 1  =  -0.9802346859567573
each class and its windows =
0 92	1 199	
 max instance in classes of test windows = 199


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.9802346859567573
max tst = 0.9484874675439612


In [14]:
len(wndws_test)

291

In [15]:
np.shape(wndws_test)

(291, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(406, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (600, 480)
y_train =>  (600,)
X_test  =>  (291, 480)
y_test  =>  (291,)
X_valid  =>  (600, 480)
y_valid  =>  (600,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
30/30 [==============================] - 3s 16ms/step - loss: 0.7240 - accuracy: 0.6283 - val_loss: 0.6205 - val_accuracy: 0.6767
Epoch 2/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.6460 - accuracy: 0.6817 - val_loss: 0.5824 - val_accuracy: 0.6867
Epoch 3/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.6231 - accuracy: 0.6633 - val_loss: 0.6721 - val_accuracy: 0.6117
Epoch 4/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.5952 - accuracy: 0.6800 - val_loss: 0.5612 - val_accuracy: 0.6933
Epoch 5/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.5892 - accuracy: 0.6900 - val_loss: 0.5682 - val_accuracy: 0.7417
Epoch 6/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.5658 - accuracy: 0.6817 - val_loss: 0.5464 - val_accuracy: 0.7317
Epoch 7/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.5608 - accuracy: 0.7167 - val_loss: 0.5401 - val_accuracy: 0.7417

Epoch 58/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.2703 - accuracy: 0.8900 - val_loss: 0.2145 - val_accuracy: 0.9200
Epoch 59/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.2815 - accuracy: 0.8900 - val_loss: 0.2849 - val_accuracy: 0.8817
Epoch 60/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.2615 - accuracy: 0.8967 - val_loss: 0.3358 - val_accuracy: 0.8483
Epoch 61/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.2696 - accuracy: 0.8917 - val_loss: 0.2068 - val_accuracy: 0.9117
Epoch 62/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.2361 - accuracy: 0.8967 - val_loss: 0.2460 - val_accuracy: 0.8850
Epoch 63/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.2652 - accuracy: 0.8883 - val_loss: 0.2288 - val_accuracy: 0.9050
Epoch 64/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.2496 - accuracy: 0.8900 - val_loss: 0.2698 - val_accuracy:

Epoch 115/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1381 - accuracy: 0.9517 - val_loss: 0.2966 - val_accuracy: 0.8783
Epoch 116/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1623 - accuracy: 0.9317 - val_loss: 0.0728 - val_accuracy: 0.9733
Epoch 117/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1694 - accuracy: 0.9333 - val_loss: 0.0997 - val_accuracy: 0.9683
Epoch 118/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1295 - accuracy: 0.9567 - val_loss: 0.2816 - val_accuracy: 0.8867
Epoch 119/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1684 - accuracy: 0.9333 - val_loss: 0.1370 - val_accuracy: 0.9467
Epoch 120/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1216 - accuracy: 0.9567 - val_loss: 0.3341 - val_accuracy: 0.8767
Epoch 121/1000
30/30 [==============================] - 0s 11ms/step - loss: 0.1587 - accuracy: 0.9533 - val_loss: 0.0781 - val_ac

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 8ms/step
array([[ 72,  20],
       [ 12, 187]], dtype=int64)
              precision    recall  f1-score   support

           1       0.86      0.78      0.82        92
           2       0.90      0.94      0.92       199

    accuracy                           0.89       291
   macro avg       0.88      0.86      0.87       291
weighted avg       0.89      0.89      0.89       291



In [27]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
30/30 [==============================] - 1s 17ms/step - loss: 0.7551 - accuracy: 0.6567 - val_loss: 0.6285 - val_accuracy: 0.6767
Epoch 2/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.6438 - accuracy: 0.6783 - val_loss: 0.7074 - val_accuracy: 0.6767
Epoch 3/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.6134 - accuracy: 0.6933 - val_loss: 0.5768 - val_accuracy: 0.6833
Epoch 4/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.6217 - accuracy: 0.6833 - val_loss: 0.5605 - val_accuracy: 0.7150
Epoch 5/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.5698 - accuracy: 0.7250 - val_loss: 0.5708 - val_accuracy: 0.6800
Epoch 6/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.5809 - accuracy: 0.7050 - val_loss: 0.5638 - val_accuracy: 0.6217
Epoch 7/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.5643 - accuracy: 0.7033 - val_loss: 0.5493 - val_accuracy: 0.7100

Epoch 58/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.3221 - accuracy: 0.8667 - val_loss: 0.2559 - val_accuracy: 0.8983
Epoch 59/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.3101 - accuracy: 0.8717 - val_loss: 0.2842 - val_accuracy: 0.8800
Epoch 60/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.3191 - accuracy: 0.8650 - val_loss: 0.2588 - val_accuracy: 0.9083
Epoch 61/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.2961 - accuracy: 0.8750 - val_loss: 0.2738 - val_accuracy: 0.8900
Epoch 62/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.3048 - accuracy: 0.8700 - val_loss: 0.2973 - val_accuracy: 0.8817
Epoch 63/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.3012 - accuracy: 0.8683 - val_loss: 0.2587 - val_accuracy: 0.8900
Epoch 64/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.2931 - accuracy: 0.8767 - val_loss: 0.2629 - val_accuracy:

Epoch 115/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.1608 - accuracy: 0.9300 - val_loss: 0.1116 - val_accuracy: 0.9567
Epoch 116/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.2183 - accuracy: 0.9350 - val_loss: 0.1230 - val_accuracy: 0.9567
Epoch 117/1000
30/30 [==============================] - 0s 12ms/step - loss: 0.1733 - accuracy: 0.9250 - val_loss: 0.2610 - val_accuracy: 0.9117
Epoch 118/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.1894 - accuracy: 0.9267 - val_loss: 0.1214 - val_accuracy: 0.9550
Epoch 119/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.1642 - accuracy: 0.9333 - val_loss: 0.1593 - val_accuracy: 0.9383
Epoch 120/1000
30/30 [==============================] - 0s 14ms/step - loss: 0.1765 - accuracy: 0.9300 - val_loss: 0.1119 - val_accuracy: 0.9583
Epoch 121/1000
30/30 [==============================] - 0s 13ms/step - loss: 0.1845 - accuracy: 0.9350 - val_loss: 0.1061 - val_ac

 max_pooling1d_6 (MaxPooling  (None, 16, 256)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 14, 512)           393728    
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 4, 512)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 2000)              4098000   
                                                                 
 dense_3 (Dense)             (None, 3)                 6003      
                                                                 
Total params: 4,818,115
Trainable params: 4,818,115
Non-trainable params: 0


In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 10ms/step
array([[ 73,  19],
       [ 12, 187]], dtype=int64)
              precision    recall  f1-score   support

           1       0.86      0.79      0.82        92
           2       0.91      0.94      0.92       199

    accuracy                           0.89       291
   macro avg       0.88      0.87      0.87       291
weighted avg       0.89      0.89      0.89       291

